In [1]:
import json
from pathlib import Path

import pandas as pd
import requests

from config import Config

In [2]:
base_dir = Path(".").resolve()
data_dir = base_dir / "data"

In [3]:
url = "https://distributor-rules.logitravelgroup.com/api/organizations/lgt/agencies/products/hot/level_commissions/new"
headers = {
    "Content-Type": "application/json",
    "Accept": "text/plain",
    "Authorization": f"ApiKey {Config.APIKEY}",
}

In [4]:
csv_file = data_dir / "pricing_rules_20241023.csv"

In [5]:
df = pd.read_csv(csv_file)

In [6]:
df[["bod_from", "bod_to", "cid_from", "cid_to"]] = df[
    ["bod_from", "bod_to", "cid_from", "cid_to"]
].apply(pd.to_datetime, errors="coerce", format="%d/%m/%Y")

In [7]:
df_grouped = (
    df.groupby(
        [
            "client_id",
            "markup_modifier",
            "bod_from",
            "bod_to",
            "cid_from",
            "cid_to",
            "dynamic",
        ],
        dropna=False,
    )
    .agg({"credential_id": lambda x: list(set(map(str, x)))})
    .reset_index()
)

In [8]:
df_grouped = df_grouped.replace({pd.NaT: None, pd.NA: None})

In [9]:
def get_date_range(date_values):
    if date_values is None:
        return {"t": 0}

    from_date, to_date = date_values
    if from_date is None or to_date is None:
        return {"t": 0}
    else:
        return {
            "t": 1,
            "f": from_date.strftime("%Y-%m-%d"),
            "u": to_date.strftime("%Y-%m-%d"),
        }

In [10]:
results_list = []
total_rows = len(df_grouped)

for index, row in df_grouped.iterrows():
    data = {
        "id": None,
        "name": f"FY_Optimization_Client_{row['client_id']}",
        "description": "FY Optimization",
        "com": row["markup_modifier"],
        "lvl": {
            "cli": {"t": 1, "l": row["credential_id"]},
            "cp": 0,
            "rat": 0,
            "prv": {"t": 0, "l": None},
            "hot": {"t": 0, "l": None},
            "mrk": {"t": 0, "l": None},
            "mel": {"t": 0, "l": None},
            "cid": get_date_range([row["cid_from"], row["cid_to"]]),
            "bod": get_date_range([row["bod_from"], row["bod_to"]]),
            "rel": 0,
            "dow": {"t": 0, "l": None},
            "age": 0,
            "room": {"t": 0, "l": None},
            "non": {"t": 0, "l": None},
            "hou": {"t": 0, "l": None},
            "isDynamicCommission": row["dynamic"],
            "ovr": False,
            "prg": {"t": 0, "f": 0, "u": 0},
            "ps": {"t": 0, "l": None},
        },
    }

    print(f"Processing row {index + 1}/{total_rows}")

    try:
        response = requests.put(url, headers=headers, json=data)

        if response.status_code == 200:
            data = response.json()["result"]
            data.update(data.pop("lvl"))

            results_list.append(data)

        elif response.status_code == 404:
            print("Error: Agencies not found.")
        else:
            print(f"Error: Received unexpected status code {response.status_code}.")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

Processing row 1/40
Processing row 2/40
Processing row 3/40
Processing row 4/40
Processing row 5/40
Processing row 6/40
Processing row 7/40
Processing row 8/40
Processing row 9/40
Processing row 10/40
Processing row 11/40
Processing row 12/40
Processing row 13/40
Processing row 14/40
Processing row 15/40
Processing row 16/40
Processing row 17/40
Processing row 18/40
Processing row 19/40
Processing row 20/40
Processing row 21/40
Processing row 22/40
Processing row 23/40
Processing row 24/40
Processing row 25/40
Processing row 26/40
Processing row 27/40
Processing row 28/40
Processing row 29/40
Processing row 30/40
Processing row 31/40
Processing row 32/40
Processing row 33/40
Processing row 34/40
Processing row 35/40
Processing row 36/40
Processing row 37/40
Processing row 38/40
Processing row 39/40
Processing row 40/40


In [11]:
metadata = {
    "editState": "edit_state",
    "isObsolete": "obsolete",
    "name": "name",
    "description": "description",
    "lastUser": "last_user",
    "lastDate": "last_date",
    "com": "commission",
    "id": "id",
    "rrg": "release_range",
    "prg": "price_range",
    "cli": "credential",
    "ps": "paying_society",
    "cp": "refundable",
    "rat": "rate",
    "prv": "provider",
    "hot": "hotel",
    "dest": "destination",
    "mrk": "market",
    "mel": "meal",
    "cid": "check_in",
    "bod": "booking_date",
    "rel": "max_release",
    "isDynamicCommission": "dynamic",
    "over": "override",
    "dow": "days_of_week",
    "age": "age",
    "room": "room",
    "non": "num_of_nights",
    "hou": "hours",
    "level_mapping": {"t": "level", "l": "list", "f": "from", "u": "to"},
}

In [12]:
for rule in results_list:
    # Rename keys according to metadata
    for key in list(rule.keys()):
        if key in metadata:
            rule[metadata[key]] = rule.pop(key)

    # Handle nested dictionary for level mapping
    level_mapping = metadata["level_mapping"]
    for key, value in rule.items():
        if isinstance(value, dict):
            for k in list(value.keys()):
                if k in level_mapping:
                    value[level_mapping[k]] = value.pop(k)

In [13]:
df_rules = pd.json_normalize(results_list, sep="_")

In [14]:
df_rules = df_rules[
    [
        "id",
        "last_user",
        "last_date",
        "name",
        "description",
        "credential_level",
        "credential_list",
        "rate",
        "provider_level",
        "provider_list",
        "hotel_level",
        "hotel_list",
        "refundable",
        "meal_level",
        "meal_list",
        "market_level",
        "market_list",
        "paying_society_level",
        "paying_society_list",
        "check_in_level",
        "check_in_from",
        "check_in_to",
        "booking_date_level",
        "booking_date_from",
        "booking_date_to",
        "commission",
        "ovr",
        "dynamic",
        "max_release",
        "days_of_week_level",
        "days_of_week_list",
        "age",
        "room_level",
        "room_list",
        "num_of_nights_level",
        "num_of_nights_list",
        "hours_level",
        "hours_list",
        "price_range_level",
        "price_range_from",
        "price_range_to",
        "release_range_level",
        "release_range_from",
        "release_range_to",
    ]
]

In [15]:
from datetime import datetime

df_rules.to_csv(
    f"pricing_rules_{datetime.today().strftime('%Y%m%d_%H%M%S')}.csv", index=False
)